# This is where I estimate baseline, naive, and ppi corrected inference parameters for each model classic, bert and gpt_zeroshot

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# load custom local packages
# sys.path.append("C:\\Users\\Adam\\Desktop\\code_projects\\GitHub\\va_nlp\\utils")
sys.path.append('/Users/adam/Desktop/Github/va_nlp/utils')

from dataset_utils import dataframe_decorator
from statistics_utils import *
from ppi_plusplus_multi import *

## load data

In [2]:
df = pd.read_csv('../src/gpt_nlp/results_df.csv')
mexico_knn = pd.read_csv('../data/results/mexico_KNN.csv')
mexico_svm = pd.read_csv('../data/results/mexico_SVM.csv')
mexico_nb = pd.read_csv('../data/results/mexico_NB.csv')
mexico_bert = pd.read_csv('../data/results/mexico_bert.csv')
mexico_gpt4zs = pd.read_csv('../data/results/mexico_gpt4_zs.csv')

In [3]:
label_to_score = {
    'aids-tb': 0,
    'communicable': 1,
    'external': 2,
    'maternal': 3, 
    'non-communicable': 4, 
    'unclassified': 'unclassified'
}

cod_labels = ['aids-tb', 'communicable', 'external', 'maternal', 'non-communicable']

# Break into 80/20 split for Naive and PPI estimation

In [4]:
def data_split(input_df, test_size=0.2):
    '''
    Takes input df which has three columns, [Y, X, Y_hat]
    Subsets df to exclude 'unclassified' from Y_hat 
    returns data split unlabeled/labeled with a default 80/20 train/test split:
        Y_sorted (ndarray): All gold-standard labels, sorted. 
        X_sorted (ndarray): All covariates corresponding to the gold-standard labels, sorted. 
        Y_lab (ndarray) : test_size number of gold standard labels.
        Yhat_lab (ndarray): test_size number of predictions corresponding to the gold-standard labels.
        X_lab (ndarray) : test_size number of covariates corresponding to the gold-standard labels.        
        Yhat_unlabeled (ndarray): (1-test_size) number of predictions corresponding to the gold-standard labels.
        X_unlabeled (ndarray): (1-test_size) number of covariates corresponding to the unlabeled data.
             
    '''
    
    # subset to drop unclassified
    subset = input_df[input_df['Y_hat']!='unclassified'].astype(int)

    # split the df into test_size (labeled) and 1-test_size (unlabeled)
    # random
    np.random.seed(123)
    labeled_df = subset.sample(frac=test_size)
    unlabeled_df = subset.drop(labeled_df.index)
    
    # separate Y's and X's
    # full Y and X
    Y = subset['Y'].to_numpy()
    X = subset['X'].to_numpy()
    X = X.reshape(-1,1)
    
    # labeled Y, X, Y_hat
    Y_lab = labeled_df['Y'].to_numpy()
    X_lab = labeled_df['X'].to_numpy()
    X_lab = X_lab.reshape(-1,1)
    Yhat_lab = labeled_df['Y_hat'].to_numpy()
    
    # unlabeled Y, X, Y_hat
    Y_unlab = unlabeled_df['Y'].to_numpy()
    X_unlab = unlabeled_df['X'].to_numpy()
    X_unlab = X_unlab.reshape(-1,1)
    Yhat_unlab = unlabeled_df['Y_hat'].to_numpy()
    
    # combine 20/80 labeled unlabeled
    X_combined = np.concatenate((X, X_unlab))
    Y_combined = np.concatenate((Y, Yhat_unlab))
    X_combined = X_combined.reshape(-1,1)
    
    # sort for MNLogit so that 0 is the left out reference category    
    sort_idx1 = np.argsort(Y)
    Y_sorted = Y[sort_idx1]
    X_sorted = X[sort_idx1]
    sort_idx2 = np.argsort(Y_lab)

    Y_combined_sorted = Y_combined[sort_idx2]
    X_combined_sorted = X_combined[sort_idx2]
    
    return Y_sorted, X_sorted, Y_lab, Yhat_lab, X_lab, Yhat_unlab, X_unlab, Y_combined_sorted, X_combined_sorted

In [5]:
def data_split(df, test_size=0.2):
    '''
    Takes input df which has three columns, [Y, X, Y_hat]
    Subsets df to exclude 'unclassified' from Y_hat 
    returns data split unlabeled/labeled with a default 80/20 train/test split:
        Y_sorted (ndarray): All gold-standard labels, sorted. 
        X_sorted (ndarray): All covariates corresponding to the gold-standard labels, sorted. 
        Y_lab (ndarray) : test_size number of gold standard labels.
        Yhat_lab (ndarray): test_size number of predictions corresponding to the gold-standard labels.
        X_lab (ndarray) : test_size number of covariates corresponding to the gold-standard labels.        
        Yhat_unlabeled (ndarray): (1-test_size) number of predictions corresponding to the gold-standard labels.
        X_unlabeled (ndarray): (1-test_size) number of covariates corresponding to the unlabeled data.
             
    '''
    
    # subset to exclude unclassified predictions
    df = df[df['Y_hat']!='unclassified'].astype(int)
    
    # random split
    np.random.seed(123)
    labeled_df = df.sample(frac=0.2)
    unlabeled_df = df.drop(labeled_df.index)
    
    Y = df['Y'].to_numpy()
    X = df['X'].to_numpy()
    X = X.reshape(-1,1)
    Yhat = df['Y_hat'].to_numpy()

    print(Y.shape)

    sort_idx1 = np.argsort(Y)
    Y_sorted = Y[sort_idx1]
    X_sorted = X[sort_idx1]
    Yhat_sorted = Yhat[sort_idx1]

    # Baseline regression ALL Y on ALL X
    mn_logit = sm.MNLogit(Y_sorted, X_sorted)
    mn_logit_res = mn_logit.fit(method = "newton", full_output = True)
    print(mn_logit_res.params)
    print(mn_logit_res.conf_int())
        
    Y_lab = labeled_df['Y'].to_numpy()
    X_lab = labeled_df['X'].to_numpy()
    X_lab = X_lab.reshape(-1,1)
    Yhat_lab = labeled_df['Y_hat'].to_numpy()
    Y_unlab = unlabeled_df['Y'].to_numpy()
    X_unlab = unlabeled_df['X'].to_numpy()
    X_unlab = X_unlab.reshape(-1,1)
    Yhat_unlab = unlabeled_df['Y_hat'].to_numpy()
        
    Y_full = np.concatenate((Y_lab, Yhat_unlab))
    X_full = np.concatenate((X_lab, X_unlab))
    
    print(Y_lab.shape)
    print(Y_full.shape)

    sort_idx2 = np.argsort(Y_full) # Sort using Y_lab or Y_full???? IS THIS CORRECT?? DOUBLE CHECK !!!!
    Y_full_sorted = Y_full[sort_idx2]
    X_full_sorted = X_full[sort_idx2]
    
    mn_logit = sm.MNLogit(Y_full_sorted, X_full_sorted)
    mn_logit_res = mn_logit.fit(method = "newton", full_output = True)
    print(mn_logit_res.params)
    print(mn_logit_res.conf_int())

#     mn_logit_res.summary()

    return Y_sorted, X_sorted, Yhat_sorted, Y_lab, X_lab, Yhat_lab, Y_unlab, X_unlab, Yhat_unlab, Y_full_sorted, X_full_sorted



In [12]:
Y_sorted, X_sorted, Yhat_sorted, Y_lab, X_lab, Yhat_lab, Y_unlab, X_unlab, Yhat_unlab, Y_full_sorted, X_full_sorted = data_split(mexico_bert)


(1306,)
Optimization terminated successfully.
         Current function value: 0.961287
         Iterations 7
[[ 0.01251579  0.00399583 -0.02938669  0.04290408]]
[[[ 0.00745418  0.0175774 ]]

 [[-0.00158093  0.00957259]]

 [[-0.03947504 -0.01929834]]

 [[ 0.03854903  0.04725912]]]
(261,)
(1306,)
Optimization terminated successfully.
         Current function value: 0.818104
         Iterations 7
[[0.01972783 0.06115925 0.01225632 0.08184039]]
[[[ 0.00652206  0.0329336 ]]

 [[ 0.04965909  0.07265941]]

 [[-0.00175172  0.02626436]]

 [[ 0.07044985  0.09323093]]]


In [13]:
# Baseline regression ALL Y on ALL X
mn_logit = sm.MNLogit(Y_sorted, X_sorted)
mn_logit_res = mn_logit.fit(method = "newton", full_output = True)
pe = mn_logit_res.params
ci = mn_logit_res.conf_int()

mn_logit_res.summary()

print(pe)
print(ci)

Optimization terminated successfully.
         Current function value: 0.961287
         Iterations 7
[[ 0.01251579  0.00399583 -0.02938669  0.04290408]]
[[[ 0.00745418  0.0175774 ]]

 [[-0.00158093  0.00957259]]

 [[-0.03947504 -0.01929834]]

 [[ 0.03854903  0.04725912]]]


In [14]:
# Naive regression using 80/20 split unlabeled/labeled
mn_logit = sm.MNLogit(Y_full_sorted, X_full_sorted)
mn_logit_res = mn_logit.fit(method = "newton", full_output = True)
pe = mn_logit_res.params
ci = mn_logit_res.conf_int()

mn_logit_res.summary()

print(pe)
print(ci)

Optimization terminated successfully.
         Current function value: 0.818104
         Iterations 7
[[0.01972783 0.06115925 0.01225632 0.08184039]]
[[[ 0.00652206  0.0329336 ]]

 [[ 0.04965909  0.07265941]]

 [[-0.00175172  0.02626436]]

 [[ 0.07044985  0.09323093]]]


In [15]:
# PPI++ correction using the same 80/20 split unlabeled/labeled
theta_ppi_ci = ppi_multiclass_logistic_ci(
            X=X_lab,
            Y=Y_lab,
            Yhat=Yhat_lab,
            X_unlabeled=X_unlab,
            Yhat_unlabeled=Yhat_unlab,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )

In [16]:
for i in range(1):
    try:
        # PPI++ correction using the same 80/20 split unlabeled/labeled
        theta_ppi_ci = ppi_multiclass_logistic_ci(
            X=X_lab,
            Y=Y_lab,
            Yhat=Yhat_lab,
            X_unlabeled=X_unlab,
            Yhat_unlabeled=Yhat_unlab,
            optimizer_options = {'disp': True, 'maxiter':1000},
        )
        print(theta_ppi_ci)
    except Exception as e:
        # Handle the exception (if needed)
        print("Error:", e)
        # Continue with the loop
        continue

{'pointest': array([ 0.01513981, -0.00346957, -0.02031198,  0.04476595]), 'ci': (array([ 0.00278088, -0.01275375, -0.0446409 ,  0.03759806]), array([0.02749875, 0.00581461, 0.00401694, 0.05193385])), 'se': array([0.0075137 , 0.00564438, 0.01479093, 0.00435777]), 'lhat': 0.016621309078555024}


In [21]:
theta_ppi_ci['ci']

(array([ 0.00278088, -0.01275375, -0.0446409 ,  0.03759806]),
 array([0.02749875, 0.00581461, 0.00401694, 0.05193385]))

NameError: name 'theta_ppi_ci' is not defined

# Site: geographic regions (mexico, ap, up, dar, bohol, pemba)
# Model: ai prediction model (classic, BERT, GPT4)
# Inference: type of inference (Baseline, Naive, PPI++)

# Loop through all permutations, compute PE and CI, save results 

In [ ]:
sites = df['site'].unique()
models = ['KNN', 'SVM', 'NB', 'bert', 'gpt4_zs']
inferences = ['baseline', 'naive', 'ppi_plus_plus']
results_list = []
column_names=  [
    'site', 'model', 'inference',
    'baseline_pe', 'baseline_lb', 'baseline_ub',
    'naive_pe', 'naive_lb', 'naive_ub',
    'ppi_plus_plus_pe', 'ppi_plus_plus_lb', 'ppi_plus_plus_ub'
]

np.random.seed(42)
for site in sites:
    for model in models:
        for inference in inferences:
            # read in dataframe for site and model
            load_df = pd.read_csv(f'../data/results/{site}_{model}.csv')
            
            # baseline predictions: Y_lab ~ X_lab
            baseline_pe = 1
            baseline_lb = 2
            baseline_ub = 3
            
            # split data 80/20 into labeled and unlabeled for naive and ppi++ inference
            X_lab, Y_lab, Yhat_lab, X_unlab, Yhat_unlab = data_split(load_df)
            
            # naive predictions: 20% Y_lab,X_lab and 80% Y_unlab,X_unlab
            naive_pe = 4
            naive_lb = 5
            naive_ub = 6
            
            # ppi++ predictions: 20% Y_lab,X_lab and 80% Y_unlab,X_unlab
            ppi_plus_plus_pe = 7
            ppi_plus_plus_lb = 8
            ppi_plus_plus_ub = 9
            
            result = [
                site, model, inference,
                baseline_pe, baseline_lb, baseline_ub,
                naive_pe, naive_lb, naive_ub,
                ppi_plus_plus_pe, ppi_plus_plus_lb, ppi_plus_plus_ub
            ]
            
            results_list.append(result)

# Create DataFrame
results_df = pd.DataFrame(results_list, columns=column_names)

# write to results folder
results_df.to_csv('../data/results/estimation_results.csv', index=False)

In [ ]:
results_df

In [ ]:
# select PE, LB and UB for given site and model and inference
site = 'mexico'
model = 'KNN'
inference = 'baseline'

results_df[(results_df['site']==site) & (results_df['model']==model)][['site',
                                                                       'model',
                                                                       'inference',
                                                                       f'{inference}_pe', 
                                                                       f'{inference}_lb', 
                                                                       f'{inference}_ub']]

# For each site/model permutation, plot the PE + CI for the three estimation procedures for each model

In [ ]:
# lhat = 0 for Classical Point Estimate
ppi_plusplus_multi.ppi_multi_class_pointestimate(
    X = X_lab,
    Y = Y_lab,
    Yhat = Yhat_lab,
    X_unlabeled = X_unlab,
    Yhat_unlabeled = Yhat_unlab,
    lhat = 0,
    coord = None,
    optimizer_options=None
)

In [ ]:
# lhat = 1 for Classical PPI Point Estimate
ppi_plusplus_multi.ppi_multi_class_pointestimate(
    X,
    Y,
    Yhat,
    X_unlabeled,
    Yhat_unlabeled,
    lhat = 1,
    coord = None,
    optimizer_options=None
)

In [ ]:
# lhat = None for PPI++ Point Estimate
ppi_plusplus_multi.ppi_multi_class_pointestimate(
    X,
    Y,
    Yhat,
    X_unlabeled,
    Yhat_unlabeled,
    lhat = 0,
    coord = None,
    optimizer_options=None
)

In [ ]:
df = pizza_eater